## Importing the required libraries for our problem

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

In [2]:
# setting the size of images
Image_Width=180
Image_Height=180
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [3]:
# loading the dataset
filenames=os.listdir("C:/Users/puja singh/Desktop/vinay/train")
categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)
#creating a dataframe of dataset
df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

## Creating a CNN model

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization

model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])

In [5]:
#getting the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 178, 178, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 89, 89, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 87, 87, 64)       2

In [6]:
# replacing the categorical values into numerical values
df["category"] = df["category"].replace({0:'cat',1:'dog'})
# split the dataset into train and test
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15

In [7]:
# rescaling the images
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "C:/Users/puja singh/Desktop/vinay/train",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "C:/Users/puja singh/Desktop/vinay/train", 
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)

test_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)

test_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "C:/Users/puja singh/Desktop/vinay/test1",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Found 0 validated image filenames belonging to 0 classes.


C:\Users\puja singh\anaconda3\envs\tensor\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 20000 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [8]:
# importing libraires for finding the best solutions for our model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [9]:
# run the model
epochs=10
history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

C:\Users\puja singh\AppData\Local\Temp\ipykernel_13440\2528005629.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
1333/1333 [==============================] - 879s 658ms/step - loss: 0.7407 - accuracy: 0.6399 - val_loss: 0.6880 - val_accuracy: 0.6342 - lr: 0.0010
Epoch 2/10
1333/1333 [==============================] - 799s 600ms/step - loss: 0.5545 - accuracy: 0.7241 - val_loss: 0.4921 - val_accuracy: 0.7552 - lr: 0.0010
Epoch 3/10
1333/1333 [==============================] - 852s 639ms/step - loss: 0.5074 - accuracy: 0.7563 - val_loss: 0.4493 - val_accuracy: 0.7986 - lr: 0.0010
Epoch 4/10
1333/1333 [==============================] - 639s 479ms/step - loss: 0.4717 - accuracy: 0.7785 - val_loss: 0.4493 - val_accuracy: 0.8016 - lr: 0.0010
Epoch 5/10
1333/1333 [==============================] - 641s 481ms/step - loss: 0.4459 - accuracy: 0.7975 - val_loss: 0.4416 - val_accuracy: 0.7964 - lr: 0.0010
Epoch 6/10
1333/1333 [==============================] - 650s 488ms/step - loss: 0.4205 - accuracy: 0.8097 - val_loss: 0.4218 - val_accuracy: 0.8192 - lr: 0.0010
Epoch 7/10
1333/1333 [============

### So our model is working fine on training dataset with 84.42% accuracy and 73.11% on validation dataset